In [6]:
!pip install PyMuPDF langchain faiss-cpu faiss-gpu torch transformers langchain-community langchain sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.2 MB/s eta 0:00:00


In [4]:
import os
import csv
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import time
import torch
from google.colab import drive

# 检查是否有可用的 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")

# 挂载 Google Drive 不适用可以去掉
# drive.mount('/content/drive')

# 1. 文档解析
def parse_pdf(pdf_dir):
    texts = []
    total_files = len([f for f in os.listdir(pdf_dir) if f.endswith('.pdf')])
    print(f"开始解析PDF文件，共{total_files}个文件")
    for i, filename in enumerate(os.listdir(pdf_dir), 1):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            print(f"正在处理第{i}/{total_files}个文件: {filename}")

            try:
                doc = fitz.open(pdf_path)
                for page in doc:
                    text = page.get_text()
                    texts.append(text)
                doc.close()
            except Exception as e:
                print(f"处理文件 {filename} 时出错: {str(e)}")

    print("PDF解析完成")
    return ' '.join(texts)

# 2. 知识库生成
def create_knowledge_base(text):
    print("开始生成知识库")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_text(text)
    print(f"文本已分割成{len(docs)}个片段")

    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5", model_kwargs={"device": device})
    print("开始创建FAISS索引")
    db = FAISS.from_texts(docs, embeddings)
    print("知识库生成完成")
    return db

# 3. 文本召回
def retrieve_answer(db, question):
    docs = db.similarity_search(question, k=1)
    return docs[0].page_content

# 主程序
def main():
    start_time = time.time()
    pdf_dir = '/content/drive/MyDrive/newwork/A/A_document'  # 更新路径
    question_file = '/content/drive/MyDrive/newwork/A/A_question.csv'  # 更新路径
    output_file = '/content/drive/MyDrive/newwork/output.csv'  # 更新路径

    # 解析PDF文档
    text = parse_pdf(pdf_dir)

    # 创建知识库
    db = create_knowledge_base(text)

    # 读取问题并检索答案
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-zh-v1.5", model_kwargs={"device": device})
    print("开始处理问题和生成答案")
    with open(question_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', newline='', encoding='utf-8') as f_out:
        reader = csv.DictReader(f_in)
        writer = csv.writer(f_out)
        writer.writerow(['ques_id', 'question', 'answer', 'embedding'])

        total_questions = sum(1 for row in reader)  # 计算总问题数
        f_in.seek(0)  # 重置文件指针
        next(reader)  # 跳过标题行

        for i, row in enumerate(reader, 1):
            ques_id = row['ques_id']
            question = row['question']
            print(f"处理问题 {i}/{total_questions}: {ques_id}")
            answer = retrieve_answer(db, question)
            embedding = embeddings.embed_query(answer)

            writer.writerow([ques_id, question, answer, embedding])

    end_time = time.time()
    print(f"程序执行完成，总耗时: {end_time - start_time:.2f} 秒")

# if __name__ == "__main__":
#     main()

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


使用设备: cuda
Mounted at /content/drive


In [7]:
main()

开始解析PDF文件，共120个文件
正在处理第1/120个文件: AF05.pdf
正在处理第2/120个文件: AF02.pdf
正在处理第3/120个文件: AF04.pdf
正在处理第4/120个文件: AF08.pdf
正在处理第5/120个文件: AF07.pdf
正在处理第6/120个文件: AF01.pdf
正在处理第7/120个文件: AF31.pdf
正在处理第8/120个文件: AF29.pdf
正在处理第9/120个文件: AF17.pdf
正在处理第10/120个文件: AF28.pdf
正在处理第11/120个文件: AF11.pdf
正在处理第12/120个文件: AF26.pdf
正在处理第13/120个文件: AF10.pdf
正在处理第14/120个文件: AF39.pdf
正在处理第15/120个文件: AF41.pdf
正在处理第16/120个文件: AF16.pdf
正在处理第17/120个文件: AF19.pdf
正在处理第18/120个文件: AF20.pdf
正在处理第19/120个文件: AF14.pdf
正在处理第20/120个文件: AF37.pdf
正在处理第21/120个文件: AF23.pdf
正在处理第22/120个文件: AF35.pdf
正在处理第23/120个文件: AF33.pdf
正在处理第24/120个文件: AF22.pdf
正在处理第25/120个文件: AF13.pdf
正在处理第26/120个文件: AF25.pdf
正在处理第27/120个文件: AT14.pdf
正在处理第28/120个文件: AT22.pdf
正在处理第29/120个文件: AT02.pdf
正在处理第30/120个文件: AT09.pdf
正在处理第31/120个文件: AT18.pdf
正在处理第32/120个文件: AT15.pdf
正在处理第33/120个文件: AT04.pdf
正在处理第34/120个文件: AT16.pdf
正在处理第35/120个文件: AT08.pdf
正在处理第36/120个文件: AT20.pdf
正在处理第37/120个文件: AT19.pdf
正在处理第38/120个文件: AT10.pdf
正在处理第39/120个文件: AT05.pdf
正在处理第40/120个文件: 

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

开始创建FAISS索引
知识库生成完成
开始处理问题和生成答案
处理问题 1/100: 1
处理问题 2/100: 2
处理问题 3/100: 3
处理问题 4/100: 4
处理问题 5/100: 5
处理问题 6/100: 6
处理问题 7/100: 7
处理问题 8/100: 8
处理问题 9/100: 9
处理问题 10/100: 10
处理问题 11/100: 11
处理问题 12/100: 12
处理问题 13/100: 13
处理问题 14/100: 14
处理问题 15/100: 15
处理问题 16/100: 16
处理问题 17/100: 17
处理问题 18/100: 18
处理问题 19/100: 19
处理问题 20/100: 20
处理问题 21/100: 21
处理问题 22/100: 22
处理问题 23/100: 23
处理问题 24/100: 24
处理问题 25/100: 25
处理问题 26/100: 26
处理问题 27/100: 27
处理问题 28/100: 28
处理问题 29/100: 29
处理问题 30/100: 30
处理问题 31/100: 31
处理问题 32/100: 32
处理问题 33/100: 33
处理问题 34/100: 34
处理问题 35/100: 35
处理问题 36/100: 36
处理问题 37/100: 37
处理问题 38/100: 38
处理问题 39/100: 39
处理问题 40/100: 40
处理问题 41/100: 41
处理问题 42/100: 42
处理问题 43/100: 43
处理问题 44/100: 44
处理问题 45/100: 45
处理问题 46/100: 46
处理问题 47/100: 47
处理问题 48/100: 48
处理问题 49/100: 49
处理问题 50/100: 50
处理问题 51/100: 51
处理问题 52/100: 52
处理问题 53/100: 53
处理问题 54/100: 54
处理问题 55/100: 55
处理问题 56/100: 56
处理问题 57/100: 57
处理问题 58/100: 58
处理问题 59/100: 59
处理问题 60/100: 60
处理问题 61/100: 61
处理问题 62/10

Python 3.10.12
